# Python Resource Management Project:

[Overview link](https://console.cloud.google.com/auth/overview?project=python-resource-mgt)

<br>

GCP Service account:  ```moneypenny@python-resource-mgt.iam.gserviceaccount.com```


### OAuth Scopes:

* Gmail:
    * ```https://www.googleapis.com/auth/gmail.modify```
    * ```https://www.googleapis.com/auth/gmail.labels```

* Sheets:
    * ```https://www.googleapis.com/auth/spreadsheets```

* Drive:
    * ```https://www.googleapis.com/auth/drive.file```

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.oauth2 import service_account
from googleapiclient.discovery import build
from datetime import datetime, timedelta
import base64
import email
import pytz
import schedule
import time
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage

In [ ]:
def create_service(service_account_file, scopes, api_name, api_version):
    """
    Create a Google service object with the specified credentials and scopes.
    
    Args:
        service_account_file (str): Path to service account JSON key file
        scopes (list): List of OAuth scopes needed
        api_name (str): Name of the Google API (e.g., 'gmail', 'sheets', 'drive')
        api_version (str): Version of the API to use
    
    Returns:
        service: Authenticated Google service object
    """
    credentials = service_account.Credentials.from_service_account_file(
        service_account_file,
        scopes=scopes
    )
    
    service = build(api_name, api_version, credentials=credentials)
    return service

# Example usage
def main():
    # Path to your service account JSON key file
    SERVICE_ACCOUNT_FILE = '/mnt/d/GCP/python-resource-mgt-0ac59ab81869.json'
    
    # Define the scopes you need
    SCOPES = [
        'https://www.googleapis.com/auth/gmail.modify',
        'https://www.googleapis.com/auth/gmail.labels',
        'https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/drive.file'
    ]
    
    # Create services for each API
    gmail_service = create_service(SERVICE_ACCOUNT_FILE, SCOPES, 'gmail', 'v1')
    sheets_service = create_service(SERVICE_ACCOUNT_FILE, SCOPES, 'sheets', 'v4')
    drive_service = create_service(SERVICE_ACCOUNT_FILE, SCOPES, 'drive', 'v3')
    
    # Now you can use these service objects to make API calls
    # Example: List files in Drive
    results = drive_service.files().list(pageSize=10).execute()
    files = results.get('files', [])
    
    for file in files:
        print(f"Found file: {file['name']} ({file['id']})")

if __name__ == '__main__':
    main()